In [2]:
!pip3 install pandas dash

  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527013 sha256=09195fa94d31b8e42a663cc23affdabde327532c792ac7c3ec669472fb25528a
  Stored in directory: c:\users\abakade\appdata\local\pip\cache\wheels\17\6f\34\e5d7fe1c7189531b3c6c90266df517f73210b9ccf6aabd9c53
  Created wheel for dash-html-components: filename=dash_html_components-1.1.2-py3-none-any.whl size=427844 sha256=bdb719bb82bc13f424609ec6d9291aecb09851126855369065cc7dc062abef5f
  Stored in directory: c:\users\abakade\appdata\local\pip\cache\wheels\89\69\97\d277cdcfc9018c0c50a181a3572895c323f324ce30616e988e
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-py3-none-any.whl size=1014868 sha256=66dfd3e478e10b695d7bd34ee96446c5ba9c6655509ca9e03708b2f489741d8d
  Stored in directory: c:\users\abakade\appdata\local\pip\cache\wheels\9d\82\83\b39e9f2d6941ed7f500c579bb8c2e20c317638d84be58c63f2
  Created wheel for dash-table: filename=dash_table-4.11.2-py3-none-any.whl size=

In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

### Read the SpaceX data into pandas dataframe

In [3]:
spacex_df = pd.read_csv('spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

### Create a dash application

In [4]:
app = dash.Dash(__name__)

### Create an app layout 

In [ ]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                         {'label': 'ALL SITES', 'value': 'ALL'},
                                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                    ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here", 
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,max=10000,step=1000,
                                                value=[min_payload,max_payload],
                                                marks={0: '0', 2500:'2500',5000:'5000',
                                                7500:'7500', 10000: '10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def build_graph(site_dropdown):
    if site_dropdown == 'ALL':
        piechart = px.pie(data_frame = spacex_df, names='Launch Site', values='class' ,title='Total Launches for All Sites')
        return piechart
    else:
        #specific_df = spacex_df['Launch Site']
        specific_df=spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        piechart = px.pie(data_frame = specific_df, names='class',title='Total Launch for a Specific Site')
        return piechart

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')])

def update_graph(site_dropdown, payload_slider):
    if site_dropdown == 'ALL':
        filtered_data = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0])
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="Payload Mass (kg)", y="class", 
        color="Booster Version Category")
        return scatterplot
    else:
        specific_df=spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        filtered_data = specific_df[(specific_df['Payload Mass (kg)']>=payload_slider[0])
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="Payload Mass (kg)", y="class", 
        color="Booster Version Category")
        return scatterplot

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jul/2022 12:18:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:18:43] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_0m1657669371.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:18:43] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_0m1657669371.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:18:43] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_0m1657669371.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:18:43] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_0m1657669371.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:18:44] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_15_0m1657669371.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:18:44] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_15_0m1657669371.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 1